Descargar el dataset y configurarlo

In [52]:
# Descarga del dataset
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn import cross_validation
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
url = 'http://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/prostate.data'
df = pd.read_csv(url, sep='\t', header=0)
df = df.drop('Unnamed: 0', axis=1)
istrain_str = df['train']
istrain = np.asarray([True if s == 'T' else False for s in istrain_str])
istest = np.logical_not(istrain)
df = df.drop('train', axis=1)

scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_scaled['lpsa'] = df['lpsa']

X = df_scaled.ix[:,:-1] #se le elimina la columna del target
N = X.shape[0]
X.insert(X.shape[1], 'intercept', np.ones(N))
y = df_scaled['lpsa'] #columna target
Xtrain = X[istrain]
ytrain = y[istrain]
Xtest = X[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]

Xm = Xtrain.as_matrix()
ym = ytrain.as_matrix()

In [4]:
# A)

# En la tercera linea se saca la columna "intercept", puesto que ya no se necesita.
# En la novena linea se ejecuta el metodo de Ridge a traves de factorizacion svd
# con el parametro fit_intercept igual a True, puesto que el mismo metodo se encarga del intercepto.

In [53]:
from sklearn.linear_model import Ridge
import matplotlib.pylab as plt
X = X.drop('intercept', axis=1)
Xtrain = X[istrain]
ytrain = y[istrain]
names_regressors = ["Lcavol", "Lweight", "Age", "Lbph", "Svi", "Lcp", "Gleason", "Pgg45"]
alphas_ = np.logspace(4,-1,base=10)
coefs = []
model = Ridge(fit_intercept=True,solver='svd')
for a in alphas_:
    model.set_params(alpha=a)
    model.fit(Xtrain, ytrain)
    coefs.append(model.coef_)
ax = plt.gca()
for y_arr, label in zip(np.squeeze(coefs).T, names_regressors): #que hace aca?
    print alphas_.shape
    print y_arr.shape
    plt.plot(alphas_, y_arr, label=label)
plt.legend()
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1]) # reverse axis
plt.xlabel('alpha')
plt.ylabel('weights')
plt.title('Regularization Path RIDGE')
plt.axis('tight')
plt.legend(loc=2)
plt.show()

(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)


modelo lasso

In [55]:
from sklearn.linear_model import Lasso
Xtrain = X[istrain]
ytrain = y[istrain]
names_regressors = ["Lcavol", "Lweight", "Age", "Lbph", "Svi", "Lcp", "Gleason", "Pgg45"]
alphas_ = np.logspace(1,-2,base=10)
coefs = []
model = Lasso(fit_intercept=True)
for a in alphas_:
	model.set_params(alpha=a)
	model.fit(Xtrain, ytrain)
	coefs.append(model.coef_)
ax = plt.gca()
for y_arr, label in zip(np.squeeze(coefs).T, names_regressors):
	print alphas_.shape
	print y_arr.shape
	plt.plot(alphas_, y_arr, label=label)
plt.legend()
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1]) # reverse axis
plt.xlabel('alpha')
plt.ylabel('weights')
plt.title('Regularization Path LASSO')
plt.axis('tight')
plt.legend(loc=2)
plt.show()

(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)
(50,)


alpha y los errores en modelo ridge

In [54]:
Xtest = X[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]
alphas_ = np.logspace(2,-2,base=10)
coefs = []
model = Ridge(fit_intercept=True)
mse_test = []
mse_train = []
for a in alphas_:
	model.set_params(alpha=a)
	model.fit(Xtrain, ytrain)
	yhat_train = model.predict(Xtrain)
	yhat_test = model.predict(Xtest)
	mse_train.append(np.mean(np.power(yhat_train - ytrain, 2)))
	mse_test.append(np.mean(np.power(yhat_test - ytest, 2)))
ax = plt.gca()
ax.plot(alphas_,mse_train,label='train error Ridge')
ax.plot(alphas_,mse_test,label='test error Ridge')
plt.legend(loc=2)
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1])
plt.show()

alpha y errores en modelo lasso

In [7]:
Xtest = X[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]
alphas_ = np.logspace(0.5,2,base=10)
coefs = []
model = Lasso(fit_intercept=True)
mse_test = []
mse_train = []
for a in alphas_:
	model.set_params(alpha=a)
	model.fit(Xtrain, ytrain)
	yhat_train = model.predict(Xtrain)
	yhat_test = model.predict(Xtest)
	mse_train.append(np.mean(np.power(yhat_train - ytrain, 2)))
	mse_test.append(np.mean(np.power(yhat_test - ytest, 2)))
ax = plt.gca()
ax.plot(alphas_,mse_train,label='train error Lasso')
ax.plot(alphas_,mse_test,label='test error Lasso')
plt.legend(loc=2)
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1])
plt.show()

mejor parametro alpha para cada modelo

In [56]:
def MSE(y,yhat): return np.mean(np.power(y-yhat,2))

Xm = Xtrain.as_matrix()
ym = ytrain.as_matrix()

def best_alpha(alphas_, Xm,ym, model):
    k_fold = cross_validation.KFold(len(Xm),10)
    best_cv_mse = float("inf")
    for a in alphas_:
        model.set_params(alpha=a)
        mse_list_k10 = [MSE(model.fit(Xm[train], ym[train]).predict(Xm[val]), ym[val]) for train, val in k_fold]
        if np.mean(mse_list_k10) < best_cv_mse:
            best_cv_mse = np.mean(mse_list_k10)
            best_alpha = a
            print "BEST PARAMETER=%f, MSE(CV)=%f"%(best_alpha,best_cv_mse) #para visualizar lo que sucede

alphas_ridge = np.logspace( 2 ,-2  , base=10) 
alphas_lasso = np.logspace( 0.5 , -1, base = 10)

#ejecucion
print "para Ridge:"
best_alpha(alphas_ridge,Xm,ym, Ridge(fit_intercept=True))
print "para Lasso:"
best_alpha(alphas_lasso,Xm,ym, Lasso(fit_intercept=True))

para Ridge:
BEST PARAMETER=100.000000, MSE(CV)=0.999646
BEST PARAMETER=82.864277, MSE(CV)=0.965780
BEST PARAMETER=68.664885, MSE(CV)=0.935286
BEST PARAMETER=56.898660, MSE(CV)=0.908007
BEST PARAMETER=47.148664, MSE(CV)=0.883727
BEST PARAMETER=39.069399, MSE(CV)=0.862205
BEST PARAMETER=32.374575, MSE(CV)=0.843206
BEST PARAMETER=26.826958, MSE(CV)=0.826516
BEST PARAMETER=22.229965, MSE(CV)=0.811949
BEST PARAMETER=18.420700, MSE(CV)=0.799351
BEST PARAMETER=15.264180, MSE(CV)=0.788588
BEST PARAMETER=12.648552, MSE(CV)=0.779537
BEST PARAMETER=10.481131, MSE(CV)=0.772077
BEST PARAMETER=8.685114, MSE(CV)=0.766076
BEST PARAMETER=7.196857, MSE(CV)=0.761390
BEST PARAMETER=5.963623, MSE(CV)=0.757863
BEST PARAMETER=4.941713, MSE(CV)=0.755328
BEST PARAMETER=4.094915, MSE(CV)=0.753618
BEST PARAMETER=3.393222, MSE(CV)=0.752571
BEST PARAMETER=2.811769, MSE(CV)=0.752036
BEST PARAMETER=2.329952, MSE(CV)=0.751881
para Lasso:
BEST PARAMETER=3.162278, MSE(CV)=1.795596
BEST PARAMETER=0.954095, MSE(CV)=1.791